In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from itertools import product
import calendar

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.style.use('seaborn')

In [3]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, recall_score, precision_score
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, RobustScaler, PowerTransformer, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.multioutput import MultiOutputRegressor
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_recall_fscore_support, f1_score

In [4]:
from model.helpers.ts_differencing import *

<a id='wrangling'></a>
## Data Wrangling

In this section we'll load the data, assess it to check for any quality or tidiness issues and clean where needed.

<a id='gather'></a>
### Gather

In [5]:
df = pd.read_csv('../data/consolidated_coin_data.csv', parse_dates=['Date'], index_col='Date')

In [6]:
df.head()

,Currency,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,,
2019-04-24,ripple,0.321114,0.321282,0.296982,0.302318,"1,517,791,002","12,698,877,293"
2019-04-23,ripple,0.323844,0.328396,0.320919,0.321222,"1,077,333,990","13,492,933,875"
2019-04-22,ripple,0.322277,0.329350,0.320237,0.323934,"1,131,094,080","13,606,823,301"
2019-04-21,ripple,0.328678,0.329627,0.318746,0.322449,"1,005,803,846","13,533,407,430"
2019-04-20,ripple,0.331871,0.333213,0.324969,0.328476,"931,570,799","13,786,384,592"


In [7]:
df = df.copy()[df["Currency"] == 'bitcoin']

<a id='assess'></a>
### Assess

In [8]:
df.head()

,Currency,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,,
2019-04-24,bitcoin,5571.51,5642.04,5418.26,5464.87,"17,048,033,399","96,530,038,144"
2019-04-23,bitcoin,5399.37,5633.80,5389.41,5572.36,"15,867,308,108","98,417,387,004"
2019-04-22,bitcoin,5312.49,5422.69,5280.28,5399.37,"14,601,631,648","95,354,207,776"
2019-04-21,bitcoin,5335.88,5359.93,5257.34,5314.53,"13,731,844,223","93,847,052,256"
2019-04-20,bitcoin,5304.16,5358.49,5295.88,5337.89,"13,169,647,522","94,248,717,444"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2188 entries, 2019-04-24 to 2013-04-28
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Currency    2188 non-null   object 
 1   Open        2188 non-null   float64
 2   High        2188 non-null   float64
 3   Low         2188 non-null   float64
 4   Close       2188 non-null   float64
 5   Volume      2188 non-null   object 
 6   Market Cap  2188 non-null   object 
dtypes: float64(4), object(3)
memory usage: 136.8+ KB


In [10]:
df.tail()

,Currency,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,,
2013-05-02,bitcoin,116.38,125.60,92.28,105.21,-,"1,168,517,495"
2013-05-01,bitcoin,139.00,139.89,107.72,116.99,-,"1,298,954,594"
2013-04-30,bitcoin,144.00,146.93,134.05,139.00,-,"1,542,813,125"
2013-04-29,bitcoin,134.44,147.49,134.00,144.54,-,"1,603,768,865"
2013-04-28,bitcoin,135.30,135.98,132.10,134.21,-,"1,488,566,728"


In [11]:
df["Volume"].value_counts()

-                 243
99,223,800          1
43,825,000          1
1,348,950,016       1
10,391,952,498      1
                 ... 
4,530,215,219       1
48,598,100          1
167,308,000         1
733,329,984         1
7,774,230           1
Name: Volume, Length: 1946, dtype: int64

- Date is not a Date Object
- Volume and Market Cap are strings and not ints
- Columns start with capital letters and have spaces
- Dates are decreasing
- Volume column has invalid caracters

<a id='clean'></a>
### Clean

**- Columns start with capital letters and have spaces**

In [12]:
df.rename(columns={col: col.lower().replace(" ", "_") for col in df.columns}, inplace=True)

In [13]:
df.head()

,currency,open,high,low,close,volume,market_cap
Date,,,,,,,
2019-04-24,bitcoin,5571.51,5642.04,5418.26,5464.87,"17,048,033,399","96,530,038,144"
2019-04-23,bitcoin,5399.37,5633.80,5389.41,5572.36,"15,867,308,108","98,417,387,004"
2019-04-22,bitcoin,5312.49,5422.69,5280.28,5399.37,"14,601,631,648","95,354,207,776"
2019-04-21,bitcoin,5335.88,5359.93,5257.34,5314.53,"13,731,844,223","93,847,052,256"
2019-04-20,bitcoin,5304.16,5358.49,5295.88,5337.89,"13,169,647,522","94,248,717,444"


**- Volume column has invalid caracters**<br>
**- Volume and Market Cap are strings and not ints**

In [14]:
def convert_to_float(string_number):
    try:
        string_number = string_number.replace(',', '')
        string_number = float(string_number)
    except AttributeError:
        pass
    return string_number
    

In [15]:
df["volume"] = df["volume"].replace('-', np.nan)

In [16]:
df["volume"] = df["volume"].apply(convert_to_float)

In [17]:
df["market_cap"] = df["market_cap"].apply(convert_to_float)

In [18]:
df.head()

,currency,open,high,low,close,volume,market_cap
Date,,,,,,,
2019-04-24,bitcoin,5571.51,5642.04,5418.26,5464.87,1.704803e+10,9.653004e+10
2019-04-23,bitcoin,5399.37,5633.80,5389.41,5572.36,1.586731e+10,9.841739e+10
2019-04-22,bitcoin,5312.49,5422.69,5280.28,5399.37,1.460163e+10,9.535421e+10
2019-04-21,bitcoin,5335.88,5359.93,5257.34,5314.53,1.373184e+10,9.384705e+10
2019-04-20,bitcoin,5304.16,5358.49,5295.88,5337.89,1.316965e+10,9.424872e+10


In [19]:
df.tail()

,currency,open,high,low,close,volume,market_cap
Date,,,,,,,
2013-05-02,bitcoin,116.38,125.60,92.28,105.21,NaN,1.168517e+09
2013-05-01,bitcoin,139.00,139.89,107.72,116.99,NaN,1.298955e+09
2013-04-30,bitcoin,144.00,146.93,134.05,139.00,NaN,1.542813e+09
2013-04-29,bitcoin,134.44,147.49,134.00,144.54,NaN,1.603769e+09
2013-04-28,bitcoin,135.30,135.98,132.10,134.21,NaN,1.488567e+09


**- Dates are decreasing**

In [20]:
df = df.copy().iloc[::-1]

In [21]:
df.head()

,currency,open,high,low,close,volume,market_cap
Date,,,,,,,
2013-04-28,bitcoin,135.30,135.98,132.10,134.21,NaN,1.488567e+09
2013-04-29,bitcoin,134.44,147.49,134.00,144.54,NaN,1.603769e+09
2013-04-30,bitcoin,144.00,146.93,134.05,139.00,NaN,1.542813e+09
2013-05-01,bitcoin,139.00,139.89,107.72,116.99,NaN,1.298955e+09
2013-05-02,bitcoin,116.38,125.60,92.28,105.21,NaN,1.168517e+09


In [22]:
df.to_csv('../data/bitcoin_data.csv')